# Devel
Development scripts, snippets, and other code.

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../..')

In [ ]:
# Global parameters.
API_URL ='http://glasslab.neurology.emory.edu:8080/api/v1'
EXAMPLE_ITEM_ID = '65088a859a8ab9ec771ba6b3'
USER = 'jvizcar'

### Example: read images from DSA

In [ ]:
# Imports
import numpy as np
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

from neurotk import login
from neurotk.girder_utils import get_tile_metadata

In [ ]:
# Authenticate client.
gc = login(API_URL, username=USER)

In [ ]:
# Get the an example image.
item = gc.getItem(EXAMPLE_ITEM_ID)
item

In [ ]:
# Get the tile metadata to read the frame info - find the DAPI frame.
tile_metadata = get_tile_metadata(gc, EXAMPLE_ITEM_ID)

print('Channel map is the important key.')
tile_metadata

In [ ]:
# Get the DAPI image.
response = gc.get(
    f"item/{EXAMPLE_ITEM_ID}/tiles/region?units=base_pixels&exact=false&" + \
    'frame=2&encoding=PNG', 
    jsonResp=False
)

img = np.array(Image.open(BytesIO(response.content)))

# Show the image.
plt.figure(figsize=(10,10))
plt.imshow(img, cmap='gray')
plt.show()

### Blob detector

In [ ]:
# Imports
import numpy as np
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
from skimage.feature import blob_dog, blob_log, blob_doh
from math import sqrt
import cv2 as cv
from neurotk import login

In [ ]:
# Authenticate client.
gc = login(API_URL, username=USER)

In [ ]:
# Pull an image to sample.
response = gc.get(
    f"item/{EXAMPLE_ITEM_ID}/tiles/region?units=base_pixels&exact=false&" + \
    'frame=2&encoding=PNG', 
    jsonResp=False
)

img = np.array(Image.open(BytesIO(response.content)))

# Subset to only a section of the image.
x, y = 2000, 2000
size = 1280
img = img[y:y+size, x:x+size]

plt.imshow(img, cmap='gray')
plt.title('Image', fontsize=16)
plt.show()

In [ ]:
# # Improve image through preprocessing.
# from skimage.filters import threshold_otsu
# from scipy.ndimage import binary_fill_holes

# # Apply otsu thresholding.
# thresh = threshold_otsu(img)

# print(f'Otsu threshold: {thresh}')
# binary = img > thresh

# fig = plt.figure(figsize=(15, 7))
# fig.add_subplot(1, 3, 1)
# plt.imshow(img, cmap='gray')

# fig.add_subplot(1, 3, 2)
# plt.imshow(binary, cmap='gray')

# # Fill holes.
# fill = binary_fill_holes(binary)

# fig.add_subplot(1, 3, 3)
# plt.imshow(fill, cmap='gray')

# plt.show()

# img = fill.astype(np.uint8) * 255

# img = cv.cvtColor(img, cv.COLOR_GRAY2RGB)



In [ ]:
# Source: https://scikit-image.org/docs/stable/auto_examples/features_detection/plot_blob.html
# Detect blobs using Laplacian of Gaussian (LoH) method.

kwargs = {'max_sigma': 30, 'num_sigma': 15, 'threshold': 0.05}
blobs = blob_log(
    img, 
    **kwargs
)
blobs[:, 2] = blobs[:, 2] * sqrt(2)

# Different of Gaussian
# blobs = blob_dog(img, max_sigma=30, threshold=.1)
# blobs[:, 2] = blobs[:, 2] * sqrt(2)

# Difference of Hessian
# blobs = blob_doh(img, max_sigma=30, threshold=.01)

print(f'Number of blobs detection: {len(blobs)}')

In [ ]:
# Draw the circles.
fig = plt.figure(figsize=(12, 5))
fig.add_subplot(1, 2, 1)
plt.imshow(img, cmap='gray')

results = img.copy()
results = cv.cvtColor(results, cv.COLOR_GRAY2RGB)

for blob in blobs:
    y, x, r = blob.astype(int)
    
    if r <= 5:
        continue
    
    x1, y1 = x - r, y - r
    x2, y2 = x + r, y + r
    results = cv.rectangle(results, (x1, y1), (x2, y2), (255, 0, 0), 3)
    
fig.add_subplot(1, 2, 2)
plt.imshow(results)
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(9, 3), sharex=True, sharey=True)
ax = axes.ravel()

for idx, (blobs, color, title) in enumerate(sequence):
    ax[idx].set_title(title)
    ax[idx].imshow(img)
    for blob in blobs:
        y, x, r = blob
        c = plt.Circle((x, y), r, color=color, linewidth=2, fill=False)
        ax[idx].add_patch(c)
    ax[idx].set_axis_off()

plt.tight_layout()
plt.show()

In [ ]:
from pandas import read_csv

temp = read_csv('/jcDataStore/Data/nft-ai-project/datasets/annotator-datasets/csvs/expert1-test.csv').iloc[0].fp

fp = '/jcDataStore/Data/nft-ai-project/datasets/annotator-datasets/images/638147207f8a5e686a527a86-x81390y42976-x0y1920.png'
from shutil import copyfile

# label_fp = '/jcDataStore/Data/nft-ai-project/datasets/annotator-datasets/labels/638147207f8a5e686a527a86-x81390y42976-x0y1920.txt'

copyfile(fp, 'img.png')

### Add blobs to images.

In [ ]:
# Imports
from glob import glob
import numpy as np
from skimage.feature import blob_log
import matplotlib.pyplot as plt
import cv2 as cv
from typing import Tuple
from math import sqrt
from tqdm.notebook import tqdm

from os import makedirs
from os.path import join

from neurotk import imread, login
from neurotk.utils import get_filename


In [ ]:
gc = login(API_URL, username=USER)

In [ ]:
def blob_detect(fp: str, kwargs: dict, r_thr: int = 5, plot: bool = False,
                figsize: Tuple[int, int] = (7, 7), save_dir: str = None) -> str:
    """Detect blobs in an image, .
    
    Args:
        fp: Filepath of image.
        kwargs: Key-word arguments passed to skimage.feature.blob_log.
        r_thr: Remove blobs with radii smaller than this value.
        plot: Plot figures if True.
        figsize: Size of figures to plot.
        save_dir: Directory to save label text files.
    
    Returns:
        The blob coordinates in string format.
    
    """
    img = imread(fp, grayscale=True)        
    h, w = img.shape[:2]
    
    if plot:
        # Draw on the image.
        plt.figure(figsize=figsize)
        plt.imshow(img, cmap='gray')
        plt.title('Image', fontsize=16)
        plt.show()
    
    print(f'Size of image: {w} x {h}.')
    blobs = blob_log(img, **kwargs)
    
    # Add radious in their column.
    blobs[:, 2] = blobs[:, 2] * sqrt(2)
    
    print(f'{len(blobs)} number of blobs detected.')
    
    # Filter the blobs: 
    if plot:   
        img = cv.cvtColor(img, cv.COLOR_GRAY2RGB)
    
    lines = ''
    
    # Filter blobs.
    blobs = [blob for blob in blobs if blob[2] > r_thr]
    
    print(f'{len(blobs)} number of blobs after radii filtering.')

    for blob in blobs:
        y, x, r = blob.astype(int)
        
        x1, y1 = x - r, y - r
        x2, y2 = x + r, y + r
        
        lines += f'0 {x / w:4f} {y / h:4f} {(x2-x1) / w:4f} {(y2-y1) / h:4f}\n'
        
        if plot:
            img = cv.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 3)
        
    if save_dir is not None:
        # Save the file.
        with open(join(save_dir, f'{get_filename(fp)}.txt'), 'w') as fh:
            fh.write(lines.strip())
        
    if plot:
        plt.figure(figsize=figsize)
        plt.imshow(img, cmap='gray')
        plt.title('Blobs', fontsize=16)
        plt.show()
        
    return lines

In [ ]:
# Get a list of images.
src_dir = '/jcDataStore/Data/NeuroTK-Dash/nuclei-detection/'
img_fps = sorted([fp for fp in glob(join(src_dir, 'images/*.png'))])

label_dir = join(src_dir, 'labels')
makedirs(label_dir, exist_ok=True)

In [ ]:
# Standard parameters.    
kwargs = {
    'max_sigma': 30, 
    'num_sigma': 15, 
    'threshold': 0.05
}

for fp in tqdm(img_fps):
    _ = blob_detect(fp, kwargs=kwargs, save_dir=label_dir, plot=False)